In [ ]:
# Import the needed libarairies

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm


from sklearn.preprocessing import StandardScaler
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [134]:
movie_mtd = pd.read_table('data/movie_metadata_CMU_IMDB.csv', sep=',')
actor_mtd = pd.read_table('data/actor_metadata_CMU.csv', sep=',')
globes_index = pd.read_csv('data/globes_award_index.csv', sep=',')
oscars_index = pd.read_csv('data/oscars_award_index.csv', sep=',')

In [137]:
# Match the dataframes data

globes_index = globes_index.rename(columns={"nominee":"actor_name"})
oscars_index = oscars_index.rename(columns={"nominee":"actor_name"})

### Create useful features for the creation of the success index

In [ ]:
df_actors = actor_mtd[['actor_name', 'birth_date', 'gender', 'height', 'nationality']].drop_duplicates(subset='actor_name').reset_index()

min_age = actor_mtd.groupby('actor_name', as_index=False)['age_at_release'].min()
min_age.rename(columns={'age_at_release': 'age_first_movie'}, inplace=True)
df_actors = df_actors.merge(min_age, on='actor_name', how='left')

max_age = actor_mtd.groupby('actor_name', as_index=False)['age_at_release'].max()
max_age.rename(columns={'age_at_release': 'age_last_movie'}, inplace=True)
df_actors = df_actors.merge(max_age, on='actor_name', how='left')

# Number of movies
count_movies = actor_mtd.groupby('actor_name', as_index=False)['title'].count()
count_movies.rename(columns={'title' : 'number_of_movies'}, inplace=True)
df_actors = df_actors.merge(count_movies, on='actor_name', how='left')

# Career length
df_actors['career_length'] = df_actors['age_last_movie'] - df_actors['age_first_movie']

In [ ]:
# Famous movies

# Names of famous movies (top 10% based on box_office)
famous_movie_df = movie_mtd.sort_values('box_office', ascending = False)
len_revenue = int(len(famous_movie_df) * 0.1)
top_10_percent_movies = famous_movie_df.head(len_revenue)
names_fam_movies = top_10_percent_movies['title'].to_list()

# Count famous movies per actor
df_famous = actor_mtd[actor_mtd['title'].isin(names_fam_movies)]
actor_famous_count = df_famous.groupby('actor_name')['title'].nunique().reset_index()

# Merge the information with the original dataframe
all_actors = pd.DataFrame({'actor_name': actor_mtd['actor_name'].unique()})
actor_famous_count_full = pd.merge(all_actors, actor_famous_count, on='actor_name', how='left')
actor_famous_count_full['title'] = actor_famous_count_full['title'].fillna(0).astype(int)
df_actors['numb_fam_movies'] = actor_famous_count_full['title']

In [52]:
df_actors.head()

,index,actor_name,birth_date,gender,height,nationality,age_first_movie,age_last_movie,number_of_movies,career_length,numb_fam_movies
0,0,Wanda De Jesus,1958.0,F,1.620,American,31.0,51.0,8.0,20.0,5
1,1,Natasha Henstridge,1974.0,F,1.780,Other,20.0,35.0,23.0,15.0,7
2,2,Ice Cube,1969.0,M,1.727,American,22.0,42.0,33.0,20.0,20
3,3,Jason Statham,1967.0,M,1.750,Other,30.0,44.0,31.0,14.0,26
4,4,Clea DuVall,1977.0,F,1.650,American,19.0,34.0,31.0,15.0,15


## Prepare data for the model: Golden globes ground truth

In [61]:
# Merge the golden globe - based index

df_actors_merged = df_actors.merge(globes_index, on = 'actor_name', how = 'left')
df_actors_merged = df_actors_merged.fillna(0) # We suppose that the actors that are no in the globes_index dataframe have 0 nominees/awards
# Problem here -> the dataset will be umbalanced (a lot of 0s)
df_actors_merged.head()

,index,actor_name,birth_date,gender,height,nationality,age_first_movie,age_last_movie,number_of_movies,career_length,numb_fam_movies,nr_wins,nr_nominations,globes_index
0,0,Wanda De Jesus,1958.0,F,1.620,American,31.0,51.0,8.0,20.0,5,0.0,0.0,0.0
1,1,Natasha Henstridge,1974.0,F,1.780,Other,20.0,35.0,23.0,15.0,7,0.0,0.0,0.0
2,2,Ice Cube,1969.0,M,1.727,American,22.0,42.0,33.0,20.0,20,0.0,0.0,0.0
3,3,Jason Statham,1967.0,M,1.750,Other,30.0,44.0,31.0,14.0,26,0.0,0.0,0.0
4,4,Clea DuVall,1977.0,F,1.650,American,19.0,34.0,31.0,15.0,15,0.0,0.0,0.0


In [62]:
print(df_actors.shape)
print(df_actors_merged.shape)

(134079, 11)
(134079, 14)


In [89]:
# Prepare data for the regression

features = ['gender','height','age_first_movie','number_of_movies','career_length','numb_fam_movies']

X = df_actors_merged[features]
X = pd.get_dummies(X)
X = X.drop(columns=['gender_0']) # to avoid redundacy (std err, z, P>|z| were all nan)
Y = df_actors_merged['globes_index']

In [99]:
# Split to test and train
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, shuffle = True)

# Standardize data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# Convert to DataFrame with aligned indices
X_train = pd.DataFrame(X_train_std, columns=X.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_std, columns=X.columns, index=X_test.index)

# Displaying the size of each sets
print(f"The training set has {X_train.shape[0]} samples (and a shape of {X_train.shape}).")
print(f"The test set has {X_test.shape[0]} samples (and a shape of {X_test.shape}).")

print(f"The target variable y for training has the shape {y_train.shape}.")
print(f"The target variable y for testing has the shape {y_test.shape}.")

The training set has 107263 samples (and a shape of (107263, 7)).
The test set has 26816 samples (and a shape of (26816, 7)).
The target variable y for training has the shape (107263,).
The target variable y for testing has the shape (26816,).


In [100]:
from sklearn.utils import shuffle
# Undersampling to balance the dataset

# Combine the training data into a DataFrame
df_train = pd.concat([X_train, y_train], axis=1)

# Separate the majority and minority classes
df_majority = df_train[df_train['globes_index'] == 0.0]
df_minority = df_train[df_train['globes_index'] != 0.0]

# Oversample the minority class
df_minority_oversampled = df_minority.sample(len(df_majority), replace=True, random_state=42)

# Combine the majority and oversampled minority classes
df_balanced = pd.concat([df_majority, df_minority_oversampled])

# Separate features and target
X_train = df_balanced.drop('globes_index', axis=1)
y_train = df_balanced['globes_index']

# Shuffle the dataset to ensure random distribution
X_train, y_train = shuffle(X_train, y_train, random_state=42)

### Ordered regression

In [101]:
# The used indices are ordered (0<1<2<3)

model = OrderedModel(y_train, X_train, distr='logit')  # The model automatically adds a constant
result = model.fit()  # Provide appropriate weights for each class)
print(result.summary())

c:\Users\gerry\anaconda3\envs\ada\Lib\site-packages\statsmodels\base\optimizer.py:737: RuntimeWarning: Maximum number of iterations has been exceeded.
  retvals = optimize.fmin(f, start_params, args=fargs, xtol=xtol,


                             OrderedModel Results                             
Dep. Variable:           globes_index   Log-Likelihood:            -2.0084e+05
Model:                   OrderedModel   AIC:                         4.017e+05
Method:            Maximum Likelihood   BIC:                         4.018e+05
Date:                Mon, 09 Dec 2024                                         
Time:                        16:52:49                                         
No. Observations:              211920                                         
Df Residuals:                  211910                                         
Df Model:                           7                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
height               0.1636      0.004     44.340      0.000       0.156       0.171
age_first_movie     -0.0276      0

c:\Users\gerry\anaconda3\envs\ada\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [102]:
# Predict on test data
y_pred_probs = result.predict(X_test)
y_pred = y_pred_probs.idxmax(axis=1)  # Take the category with the highest probability

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy on Test Data: {accuracy:.4f}')

Accuracy on Test Data: 0.8525


In [103]:
# The dataset is umbalanced

print(f'Train set: {y_train.value_counts()}')
print(f'Test set: {y_test.value_counts()}')

Train set: globes_index
0.0    105960
1.0     61899
2.0     31898
3.0     12163
Name: count, dtype: int64
Test set: globes_index
0.0    26480
1.0      206
2.0       97
3.0       33
Name: count, dtype: int64


### Inference

In [122]:
# Predict on all data

X_new = df_actors[features]

# Pre process in the same way
X_new = pd.get_dummies(X_new)
X_new_std = scaler.transform(X_new)
X_new = pd.DataFrame(X_new_std, columns=X_new.columns, index=X_new.index)
y_pred_probs_new = result.predict(X_new)
y_pred_new = y_pred_probs_new.idxmax(axis=1)

C:\Users\gerry\AppData\Local\Temp\ipykernel_19180\3867136764.py:10: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  y_pred_new = y_pred_probs_new.idxmax(axis=1)


In [133]:
# Add the predictions to the dataframe
df_actors['predicted'] = y_pred_new

# See the average index for American vs. Foreign actors
avg_index = df_actors.groupby('nationality')['predicted'].mean()
print(f'The average success indices are: \n{avg_index}')

The average success indices are: 
nationality
American    1.082310
Other       1.046512
Name: predicted, dtype: float64


## Prepare data for the model: Oscars ground truth

In [138]:
# Merge the golden globe - based index

df_actors_merged_oscar = df_actors.merge(oscars_index, on = 'actor_name', how = 'left')
df_actors_merged_oscar = df_actors_merged_oscar.fillna(0) # We suppose that the actors that are no in the globes_index dataframe have 0 nominees/awards
# Problem here -> the dataset will be umbalanced (a lot of 0s)
df_actors_merged_oscar.head()

,index,actor_name,birth_date,gender,height,nationality,age_first_movie,age_last_movie,number_of_movies,career_length,numb_fam_movies,predicted,nr_wins,nr_nominations,oscar_index
0,0,Wanda De Jesus,1958.0,F,1.620,American,31.0,51.0,8.0,20.0,5,1.0,0.0,0.0,0.0
1,1,Natasha Henstridge,1974.0,F,1.780,Other,20.0,35.0,23.0,15.0,7,1.0,0.0,0.0,0.0
2,2,Ice Cube,1969.0,M,1.727,American,22.0,42.0,33.0,20.0,20,1.0,0.0,0.0,0.0
3,3,Jason Statham,1967.0,M,1.750,Other,30.0,44.0,31.0,14.0,26,2.0,0.0,0.0,0.0
4,4,Clea DuVall,1977.0,F,1.650,American,19.0,34.0,31.0,15.0,15,1.0,0.0,0.0,0.0


In [139]:
# Prepare data for the regression

X_oscar = df_actors_merged_oscar[features]
X_oscar = pd.get_dummies(X_oscar)
X_oscar = X_oscar.drop(columns=['gender_0']) # to avoid redundacy (std err, z, P>|z| were all nan)
Y_oscar = df_actors_merged_oscar['oscar_index']

In [140]:
# Split to test and train
X_train_oscar, X_test_oscar, y_train_oscar, y_test_oscar = train_test_split(X_oscar, Y_oscar, test_size = 0.2, random_state = 42, shuffle = True)

# Standardize data
scaler = StandardScaler()
X_train_std_oscar = scaler.fit_transform(X_train_oscar)
X_test_std_oscar = scaler.transform(X_test_oscar)

# Convert to DataFrame with aligned indices
X_train_oscar = pd.DataFrame(X_train_std_oscar, columns=X_oscar.columns, index=X_train_oscar.index)
X_test_oscar = pd.DataFrame(X_test_std_oscar, columns=X_oscar.columns, index=X_test_oscar.index)

# Displaying the size of each sets
print(f"The training set has {X_train_oscar.shape[0]} samples (and a shape of {X_train_oscar.shape}).")
print(f"The test set has {X_test_oscar.shape[0]} samples (and a shape of {X_test_oscar.shape}).")

print(f"The target variable y for training has the shape {y_train_oscar.shape}.")
print(f"The target variable y for testing has the shape {y_test_oscar.shape}.")

The training set has 107263 samples (and a shape of (107263, 7)).
The test set has 26816 samples (and a shape of (26816, 7)).
The target variable y for training has the shape (107263,).
The target variable y for testing has the shape (26816,).


In [141]:
from sklearn.utils import shuffle
# Undersampling to balance the dataset

# Combine the training data into a DataFrame
df_train_oscar = pd.concat([X_train_oscar, y_train_oscar], axis=1)

# Separate the majority and minority classes
df_majority_oscar = df_train_oscar[df_train_oscar['oscar_index'] == 0.0]
df_minority_oscar = df_train_oscar[df_train_oscar['oscar_index'] != 0.0]

# Oversample the minority class
df_minority_oversampled_oscar = df_minority_oscar.sample(len(df_majority_oscar), replace=True, random_state=42)

# Combine the majority and oversampled minority classes
df_balanced_oscar = pd.concat([df_majority_oscar, df_minority_oversampled_oscar])

# Separate features and target
X_train_oscar = df_balanced_oscar.drop('oscar_index', axis=1)
y_train_oscar = df_balanced_oscar['oscar_index']

# Shuffle the dataset to ensure random distribution
X_train_oscar, y_train_oscar = shuffle(X_train_oscar, y_train_oscar, random_state=42)

In [142]:
model_oscar = OrderedModel(y_train_oscar, X_train_oscar, distr='logit')  # The model automatically adds a constant
result_oscar = model_oscar.fit()  # Provide appropriate weights for each class)
print(result_oscar.summary())

c:\Users\gerry\anaconda3\envs\ada\Lib\site-packages\statsmodels\base\optimizer.py:737: RuntimeWarning: Maximum number of iterations has been exceeded.
  retvals = optimize.fmin(f, start_params, args=fargs, xtol=xtol,


                             OrderedModel Results                             
Dep. Variable:            oscar_index   Log-Likelihood:            -1.7539e+05
Model:                   OrderedModel   AIC:                         3.508e+05
Method:            Maximum Likelihood   BIC:                         3.509e+05
Date:                Mon, 09 Dec 2024                                         
Time:                        17:56:19                                         
No. Observations:              213096                                         
Df Residuals:                  213086                                         
Df Model:                           7                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
height              -0.2174      0.004    -52.281      0.000      -0.226      -0.209
age_first_movie     -0.0668      0

c:\Users\gerry\anaconda3\envs\ada\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [143]:
# Predict on test data
y_pred_probs_oscar = result_oscar.predict(X_test_oscar)
y_pred_oscar = y_pred_probs_oscar.idxmax(axis=1)  # Take the category with the highest probability

# Evaluate model performance
accuracy_oscar = accuracy_score(y_test_oscar, y_pred_oscar)
print(f'Accuracy on Test Data: {accuracy_oscar:.4f}')

Accuracy on Test Data: 0.6631
